In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torch.optim import Adam

In [ ]:
train_dir = "/content/drive/MyDrive/CAT/train"
val_dir   = "/content/drive/MyDrive/CAT/val"
test_dir  = "/content/drive/MyDrive/CAT/test"

In [ ]:
train_tf = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
])

val_tf = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
])

In [ ]:
train_ds = datasets.ImageFolder(train_dir, transform=train_tf)
val_ds   = datasets.ImageFolder(val_dir, transform=val_tf)
test_ds  = datasets.ImageFolder(test_dir, transform=val_tf)

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=16, shuffle=False)
test_loader  = DataLoader(test_ds, batch_size=16, shuffle=False)


In [ ]:
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 3)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 157MB/s]


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.0001)


In [ ]:
EPOCHS = 12
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0
    correct = 0
    total = 0

    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        out = model(imgs)
        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        pred = torch.argmax(out, dim=1)
        correct += (pred == labels).sum().item()
        total += labels.size(0)

    acc = correct / total
    print(f"Epoch {epoch+1}/{EPOCHS}  Loss={running_loss/len(train_loader):.3f}  Acc={acc:.3f}")


Epoch 1/12  Loss=0.759  Acc=0.660
Epoch 2/12  Loss=0.375  Acc=0.855
Epoch 3/12  Loss=0.212  Acc=0.925
Epoch 4/12  Loss=0.156  Acc=0.944
Epoch 5/12  Loss=0.133  Acc=0.963
Epoch 6/12  Loss=0.111  Acc=0.958
Epoch 7/12  Loss=0.104  Acc=0.961
Epoch 8/12  Loss=0.085  Acc=0.964
Epoch 9/12  Loss=0.106  Acc=0.963
Epoch 10/12  Loss=0.098  Acc=0.969
Epoch 11/12  Loss=0.052  Acc=0.980
Epoch 12/12  Loss=0.051  Acc=0.985


In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(device), labels.to(device)

        outputs = model(imgs)
        pred = torch.argmax(outputs, dim=1)
        correct += (pred == labels).sum().item()
        total += labels.size(0)

test_acc = correct / total
print("TEST Accuracy =", test_acc)


TEST Accuracy = 0.8547008547008547


In [ ]:


model.eval()
correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in val_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

val_acc = correct / total
print(f"Validation Accuracy = {val_acc:.4f}")


Validation Accuracy = 0.8688


In [ ]:
torch.save(model.state_dict(), "/content/drive/MyDrive/CAT/model")
print("Model training complete & saved!")

Model training complete & saved!


In [ ]:
from PIL import Image
import torch
from torchvision import transforms, models
import torch.nn as nn

classes = ["angry", "happy", "sad"]

model = models.resnet18(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, 3)
model.load_state_dict(torch.load("/content/drive/MyDrive/CAT/model", map_location="cpu"))
model.eval()

tf = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
])

def predict(img_path):
    img = Image.open(img_path).convert("RGB")
    x = tf(img).unsqueeze(0)

    with torch.no_grad():
        out = model(x)
    label = torch.argmax(out, dim=1).item()
    return classes[label]

print(predict("/content/drive/MyDrive/CAT/test/happy/1047_jpg.rf.b72d660338fc0ea07b6f5136b64e8a8b.jpg"))


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


happy


In [ ]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
!pip install efficientnet_pytorch
from efficientnet_pytorch import EfficientNet

train_dir = "/content/drive/MyDrive/CAT/train"
val_dir   = "/content/drive/MyDrive/CAT/val"
test_dir  = "/content/drive/MyDrive/CAT/test"

train_tf = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
    transforms.ToTensor(),
])

val_tf = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
])


train_ds = datasets.ImageFolder(train_dir, transform=train_tf)
val_ds   = datasets.ImageFolder(val_dir, transform=val_tf)
test_ds  = datasets.ImageFolder(test_dir, transform=val_tf)

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=16, shuffle=False)
test_loader  = DataLoader(test_ds, batch_size=16, shuffle=False)

model = EfficientNet.from_pretrained('efficientnet-b0')
model._fc = nn.Linear(model._fc.in_features, 3)  # 3 classes: happy/angry/sad

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)


criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=1e-4)


EPOCHS = 15

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0
    correct = 0
    total = 0

    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_acc = correct / total
    print(f"Epoch {epoch+1}/{EPOCHS}  Loss={running_loss/len(train_loader):.3f}  Train Acc={train_acc:.3f}")


    # Validation Accuracy

    model.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            preds = torch.argmax(outputs, dim=1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)
    val_acc = val_correct / val_total
    print(f"Validation Acc={val_acc:.3f}\n")
    model.train()


torch.save(model.state_dict(), "/content/drive/MyDrive/CAT/cat_emotion_efficientnet.pth")
print("EfficientNet-B0 model saved as cat_emotion_efficientnet.pth")


# TEST ACCURACY

model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        preds = torch.argmax(outputs, dim=1)
        test_correct += (preds == labels).sum().item()
        test_total += labels.size(0)
test_acc = test_correct / test_total
print(f"TEST Accuracy = {test_acc:.4f}")

Loaded pretrained weights for efficientnet-b0
Epoch 1/15  Loss=1.000  Train Acc=0.511
Validation Acc=0.613

Epoch 2/15  Loss=0.685  Train Acc=0.756
Validation Acc=0.753

Epoch 3/15  Loss=0.452  Train Acc=0.832
Validation Acc=0.849

Epoch 4/15  Loss=0.315  Train Acc=0.891
Validation Acc=0.877

Epoch 5/15  Loss=0.218  Train Acc=0.926
Validation Acc=0.901

Epoch 6/15  Loss=0.151  Train Acc=0.948
Validation Acc=0.886

Epoch 7/15  Loss=0.136  Train Acc=0.957
Validation Acc=0.886

Epoch 8/15  Loss=0.133  Train Acc=0.953
Validation Acc=0.905

Epoch 9/15  Loss=0.102  Train Acc=0.971
Validation Acc=0.908

Epoch 10/15  Loss=0.085  Train Acc=0.974
Validation Acc=0.901

Epoch 11/15  Loss=0.084  Train Acc=0.972
Validation Acc=0.905

Epoch 12/15  Loss=0.085  Train Acc=0.966
Validation Acc=0.910

Epoch 13/15  Loss=0.083  Train Acc=0.972
Validation Acc=0.892

Epoch 14/15  Loss=0.069  Train Acc=0.979
Validation Acc=0.899

Epoch 15/15  Loss=0.074  Train Acc=0.977
Validation Acc=0.914

EfficientNet-B0 mo